# Document Processing with Vertector

This notebook demonstrates document processing capabilities including:
- PDF, DOCX, PPTX, XLSX processing
- Classic vs VLM pipeline selection
- Export to multiple formats
- Table extraction
- OCR configuration

## Setup and Imports

In [1]:
from pathlib import Path
from vertector_data_ingestion import (
    UniversalConverter,
    LocalMpsConfig,
    CloudGpuConfig,
    CloudCpuConfig,
    ExportFormat,
    PipelineType,
    HardwareDetector,
    setup_logging,
)

# Setup logging
setup_logging(log_level="INFO")

2025-12-31 15:57:29 | INFO     | vertector_data_ingestion.monitoring.logger:setup_logging:52 - Logging initialized at INFO level


## Hardware Detection

First, let's detect available hardware to optimize our configuration.

In [2]:
# Detect hardware
hw_info = HardwareDetector.get_device_info()

print("Hardware Information:")
print("=" * 50)
for key, value in hw_info.items():
    print(f"{key}: {value}")

# Get optimized config
hw_config = HardwareDetector.detect()
print(f"\nRecommended device: {hw_config.device_type}")

2025-12-31 15:57:49 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support
2025-12-31 15:57:49 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support


Hardware Information:
device_type: mps
batch_size: 8
use_fp16: False
num_workers: 4
use_mlx: True
platform: darwin
chip: M1

Recommended device: HardwareType.MPS


## Basic Document Conversion

Convert a document using default settings.

In [3]:
# Initialize converter with hardware-optimized config
config = LocalMpsConfig()  # or CloudGpuConfig() or CloudCpuConfig()
converter = UniversalConverter(config)

# Convert a document (replace with your file)
# Note: For audio files (.wav, .mp3), install the 'asr' extra: uv sync --extra asr
doc_path = Path("../test_documents/arxiv_sample.pdf")

2025-12-31 15:57:58 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support
2025-12-31 15:57:58 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support
2025-12-31 15:57:58 | INFO     | vertector_data_ingestion.core.pipeline_router:__init__:57 - Hardware detected: mps
2025-12-31 15:57:58 | INFO     | vertector_data_ingestion.core.universal_converter:__init__:45 - Initialized UniversalConverter on mps
2025-12-31 15:57:58 | INFO     | vertector_data_ingestion.core.universal_converter:_ensure_models_available:68 - Checking model availability...


In [4]:
if doc_path.exists():
    # Use unified convert() method
    doc = converter.convert(doc_path)
    
    print(f"Document: {doc.metadata.source_path.name}")
    print(f"Pages: {doc.metadata.num_pages}")
    print(f"Pipeline: {doc.metadata.pipeline_type}")
    print(f"Processing time: {doc.metadata.processing_time:.2f}s")
else:
    print(f"File not found: {doc_path}")
    print("\nReplace with path to your PDF, DOCX, PPTX, or other document.")

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


2025-12-31 15:59:02 | INFO     | vertector_data_ingestion.core.pipeline_router:determine_pipeline:100 - Using Classic pipeline for PDF with tables: arxiv_sample.pdf
2025-12-31 15:59:02 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:180 - Converting arxiv_sample.pdf with classic pipeline
2025-12-31 15:59:02,610 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-31 15:59:02,697 - INFO - Going to convert document batch...
2025-12-31 15:59:02,698 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 870e160bad93d15722a8ae8d62725e09
2025-12-31 15:59:02,737 - INFO - Loading plugin 'docling_defaults'
2025-12-31 15:59:02,740 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-31 15:59:02,748 - INFO - Loading plugin 'docling_defaults'
2025-12-31 15:59:02,761 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-12-31 15:59:12,081 - INFO - Loading plugin 'docling_def

Document: arxiv_sample.pdf
Pages: 9
Pipeline: classic
Processing time: 121.70s


## Export to Different Formats

In [5]:
if doc_path.exists():
    # Export to Markdown
    markdown = converter.export(doc, ExportFormat.MARKDOWN)
    print("Markdown Output (first 500 chars):")
    print("=" * 50)
    print(markdown[:500])
    print("...\n")
    
    # Save to file - output_dir is automatically managed
    output_path = converter.convert_and_export(
        source=doc_path,
        output_name="document.md",
        format=ExportFormat.MARKDOWN
    )
    print(f"Saved to: {output_path}")

Markdown Output (first 500 chars):
## DocLayNet: A Large Human-Annotated Dataset for Document-Layout Analysis

Birgit Pfitzmann IBM Research Rueschlikon, Switzerland bpf@zurich.ibm.com

Christoph Auer IBM Research Rueschlikon, Switzerland cau@zurich.ibm.com

Ahmed S. Nassar IBM Research

Rueschlikon, Switzerland ahn@zurich.ibm.com

Michele Dolfi IBM Research Rueschlikon, Switzerland dol@zurich.ibm.com

Peter Staar IBM Research Rueschlikon, Switzerland taa@zurich.ibm.com

Figure 1: Four examples of complex page layouts across diff
...



2025-12-31 16:00:41 | INFO     | vertector_data_ingestion.core.pipeline_router:determine_pipeline:100 - Using Classic pipeline for PDF with tables: arxiv_sample.pdf
2025-12-31 16:00:41 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:180 - Converting arxiv_sample.pdf with classic pipeline
2025-12-31 16:00:41,831 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-31 16:00:41,836 - INFO - Going to convert document batch...
2025-12-31 16:00:41,836 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 870e160bad93d15722a8ae8d62725e09
2025-12-31 16:00:41,836 - INFO - Accelerator device: 'mps'
2025-12-31 16:00:42,408 - INFO - Accelerator device: 'mps'
2025-12-31 16:00:43,006 - INFO - Processing document arxiv_sample.pdf
2025-12-31 16:00:54,825 - INFO - Finished converting document arxiv_sample.pdf in 13.00 sec.
2025-12-31 16:00:54 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:199 - Converted 

Saved to: output/document.md


## Pipeline Selection

Compare Classic vs VLM pipelines.

In [7]:
import time

if doc_path.exists():
    print("Classic Pipeline:")
    start = time.time()
    classic_doc = converter.convert(doc_path, use_vlm=False)
    classic_time = time.time() - start
    print(f"  Time: {classic_time:.2f}s")
    
    print("\nVLM Pipeline:")
    start = time.time()
    vlm_doc = converter.convert(doc_path, use_vlm=True)
    vlm_time = time.time() - start
    print(f"  Time: {vlm_time:.2f}s")
    print(f"\nVLM is {vlm_time/classic_time:.2f}x slower (but more accurate)")

2025-12-31 16:05:14 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:180 - Converting arxiv_sample.pdf with classic pipeline
2025-12-31 16:05:14,310 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-31 16:05:14,316 - INFO - Going to convert document batch...
2025-12-31 16:05:14,317 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 870e160bad93d15722a8ae8d62725e09
2025-12-31 16:05:14,318 - INFO - Accelerator device: 'mps'


Classic Pipeline:


2025-12-31 16:05:15,260 - INFO - Accelerator device: 'mps'
2025-12-31 16:05:15,740 - INFO - Processing document arxiv_sample.pdf
2025-12-31 16:05:28,163 - INFO - Finished converting document arxiv_sample.pdf in 13.85 sec.
2025-12-31 16:05:28 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:199 - Converted arxiv_sample.pdf in 13.86s (9 pages, 0.6 pages/sec)
2025-12-31 16:05:28 | INFO     | vertector_data_ingestion.core.pipeline_router:_build_vlm_options:233 - Using Granite-Docling-MLX (258M params, default)
2025-12-31 16:05:28 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:180 - Converting arxiv_sample.pdf with vlm pipeline
2025-12-31 16:05:28,173 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-31 16:05:28,177 - INFO - Going to convert document batch...
2025-12-31 16:05:28,178 - INFO - Initializing pipeline for VlmPipeline with options hash 40c755e91ed89bcff77c660f7fe19c81


  Time: 13.87s

VLM Pipeline:


2025-12-31 16:05:28,858 - INFO - Processing document arxiv_sample.pdf
2025-12-31 16:08:18,440 - INFO - Finished converting document arxiv_sample.pdf in 170.27 sec.
2025-12-31 16:08:18 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:199 - Converted arxiv_sample.pdf in 170.27s (9 pages, 0.1 pages/sec)


  Time: 170.27s

VLM is 12.28x slower (but more accurate)


## Batch Processing

Process multiple documents with the same unified API.

In [9]:
documents_dir = Path("../test_documents/")

if documents_dir.exists():
    pdf_files = list(documents_dir.glob("*.pdf"))[:5]
    
    if pdf_files:
        print(f"Processing {len(pdf_files)} documents...")
        
        # Same convert() method - just pass a list!
        results = converter.convert(pdf_files, parallel=True)
        
        print("\nResults:")
        for doc in results:
            print(f"  {doc.metadata.source_path.name}: {doc.metadata.num_pages} pages")
    else:
        print("No PDF files found")
else:
    print("Create a 'documents/' directory with PDFs to test batch processing")

2025-12-31 16:09:33 | INFO     | vertector_data_ingestion.core.universal_converter:convert_batch:331 - Converting 3 documents in parallel
2025-12-31 16:09:36 | INFO     | vertector_data_ingestion.core.pipeline_router:determine_pipeline:97 - Using VLM pipeline for scanned PDF: ocr_test.pdf


Processing 3 documents...


2025-12-31 16:09:36 | INFO     | vertector_data_ingestion.core.pipeline_router:determine_pipeline:106 - Using Classic pipeline (default) for 2112.13734v2.pdf
2025-12-31 16:09:36 | INFO     | vertector_data_ingestion.core.pipeline_router:_build_vlm_options:233 - Using Granite-Docling-MLX (258M params, default)
2025-12-31 16:09:36 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:180 - Converting 2112.13734v2.pdf with classic pipeline
2025-12-31 16:09:36 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:180 - Converting ocr_test.pdf with vlm pipeline
2025-12-31 16:09:37,055 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-31 16:09:37,098 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-31 16:09:37,149 - INFO - Going to convert document batch...
2025-12-31 16:09:37,191 - INFO - Going to convert document batch...
2025-12-31 16:09:37,252 - INFO - Initializing pipeline for StandardPdfPipeline with opt


Results:
  ocr_test.pdf: 1 pages
  2112.13734v2.pdf: 4 pages
  arxiv_sample.pdf: 9 pages


## Summary

This notebook demonstrated:
- Unified `convert()` API for single and batch processing
- Hardware detection
- Pipeline selection
- Export capabilities

Next: Check out `02_audio_transcription.ipynb`